In [1]:
import pandas as pd

In [2]:
# test set

In [3]:
df = pd.read_csv('/mnt/green-efs/oleksandr.korniienko/data/nucle.test.csv')
df

,src
0,Keeping the Secret of Genetic Testing
1,What is genetic risk ?
2,Genetic risk refers more to your chance of inh...
3,People get certain disease because of genetic ...
4,How much a genetic change tells us about your ...
...,...
1307,"As a result , if the situation keep go on in t..."
1308,"To put it in the nutshell , everything has its..."
1309,So does social media network .
1310,It not only is convenient and beneficial in co...


In [9]:
# samples = []
# instruction = "Rewrite this text to make it grammatically correct ."
# for _, row in df.iterrows():
#     sample =   {
#         "system": ,
#         "instruction": "",
#         "input": row.src,
#         "output": "",
#     }
#     samples.append(sample)
# samples[0]

{'system': 'You are helpful AI writing assistant.',
 'instruction': 'Rewrite this text to make it grammatically correct .',
 'input': 'Keeping the Secret of Genetic Testing',
 'output': ''}

In [23]:
prompt = "'<s>[INST] <<SYS>>\nRewrite this text to make it grammatically correct .\n<</SYS>>\n\n{text} [/INST]'"

samples = []
for _, row in df.iterrows():
    sample =   {
        "instruction": "Rewrite this text to make it grammatically correct .",
        "input": row.src,
        "output":"",
    }
    samples.append(sample)
samples[0]


{'instruction': 'Rewrite this text to make it grammatically correct .',
 'input': 'Keeping the Secret of Genetic Testing',
 'output': ''}

In [24]:
import json
with open('../gec_nucle_test.json', 'w') as f:
    json.dump(samples, f, indent=2)


In [26]:
import spacy
NLP = spacy.load('en_core_web_sm')

import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

def tokenize(txt):
    doc = NLP(txt)
    return " ".join([t.text for t in doc])



[nltk_data] Downloading package punkt to
[nltk_data]     /home/oleksandr.korniienko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
# Gemma-7B-chat-gec  LLaMA2-7B-chat-gec  LLaMA2-7B-gec  Mistral-7B-Instruct-gec  phi-2-gec
model_name = 'Gemma-7B-chat-gec'
model_name = 'LLaMA2-7B-chat-gec'


data_path = f'/home/oleksandr.korniienko/LLaMA-Factory/saves/{model_name}/lora-all/predict/generated_predictions.jsonl'
output_path = f'/home/oleksandr.korniienko/LLaMA-Factory/saves/{model_name}/lora-all/predict/output.csv'

import json
with open(data_path, 'r') as f:
    data = [json.loads(l) for l in f.readlines()]

df_output = pd.DataFrame(data)

df = pd.read_csv('/mnt/green-efs/oleksandr.korniienko/data/nucle.test.csv')

df['output'] = df_output['predict']

df['output'] = df.output.map(tokenize)

df.to_csv(output_path, index=False)

output_path

'/home/oleksandr.korniienko/LLaMA-Factory/saves/LLaMA2-7B-chat-gec/lora-all/predict/output.csv'

In [36]:
cmd = f's2s_evaluate --eval /mnt/green-efs/kostiantyn.omelianchuk/data/evalsets/nucle14-2a.m2 --pred {output_path}  --metrics max_match wed --print-format yaml'

cmd

's2s_evaluate --eval /mnt/green-efs/kostiantyn.omelianchuk/data/evalsets/nucle14-2a.m2 --pred /home/oleksandr.korniienko/LLaMA-Factory/saves/LLaMA2-7B-chat-gec/lora-all/predict/output.csv  --metrics max_match wed --print-format yaml'